# Stripe Financial Connections Setup

Quick setup to connect your test bank accounts!

Steps:
1. Install dependencies
2. Load API keys
3. Create session and connect accounts via browser
4. Test the connection

## Step 1: Install & Load

In [ ]:
# Install dependencies
%pip install -q -r ../requirements.txt

import os
import stripe
from dotenv import load_dotenv

# Load environment variables
load_dotenv("../secrets.env")
load_dotenv("../variables.env")

# Get Stripe API key
stripe_key = os.getenv("STRIPE_API_KEY")

if not stripe_key or not stripe_key.startswith("sk_test_"):
    print("❌ Missing or invalid STRIPE_API_KEY in secrets.env")
    print("   Must start with sk_test_")
else:
    stripe.api_key = stripe_key
    print("✅ Stripe API key loaded!")

## Step 2: Create Session & Connect Accounts

This creates a session and gives you a URL to connect test bank accounts.

In [ ]:
print("Creating Financial Connections Session...\n")

# Create a test customer
customer = stripe.Customer.create(
    name="Test User",
    email="test@example.com",
)
print(f"✅ Customer: {customer.id}")

# Create Financial Connections session
session = stripe.financial_connections.Session.create(
    account_holder={
        "type": "customer",
        "customer": customer.id,
    },
    permissions=["balances", "transactions", "payment_method"],
    filters={"countries": ["US"]},
)

STRIPE_SESSION_ID = session.id

print(f"\n" + "="*60)
print("📋 SAVE THIS SESSION ID")
print("="*60)
print(f"\n{STRIPE_SESSION_ID}\n")

# For browser auth, open stripe_auth.html and use:
print("\n🔗 To connect accounts:")
print(f"1. Open stripe_auth.html in your browser")
print(f"2. Paste your publishable key (pk_test_...)")
print(f"3. Paste this client secret:")
print(f"   {session.client_secret}")
print(f"\n4. In the auth flow:")
print(f"   - Select 'Test Institution' or 'Sandbox Bank'")
print(f"   - Enter ANY username/password")
print(f"   - Select test accounts")
print(f"\n5. Come back here and run the next cell!")

## Step 3: Verify Accounts Connected

In [ ]:
# If you already have a session ID, uncomment and paste it here:
# STRIPE_SESSION_ID = "fcsess_..."

print("Checking connected accounts...\n")

session = stripe.financial_connections.Session.retrieve(STRIPE_SESSION_ID)

if not session.accounts or len(session.accounts.data) == 0:
    print("⚠️  No accounts connected yet")
    print("   Complete the auth flow from the previous cell first!")
else:
    print(f"✅ Found {len(session.accounts.data)} connected accounts!\n")
    
    for account in session.accounts.data:
        full_account = stripe.financial_connections.Account.retrieve(account.id)
        print(f"  📊 {full_account.display_name}")
        print(f"     Type: {full_account.subcategory}")
        print(f"     Last 4: {full_account.last4}")
        print()
    
    print("\n" + "="*60)
    print("💾 Add to secrets.env:")
    print("="*60)
    print(f"STRIPE_SESSION_ID={STRIPE_SESSION_ID}")
    print("\nNow run the next cell to test with balances!")

## Step 4: Test with Real Balances

This refreshes balances and tests the full integration.

In [ ]:
import sys
sys.path.append('..')
from stripe_integration.client import StripeFinancialClient

print("Testing Stripe Financial Client...\n")

# Create client
client = StripeFinancialClient()

# Get accounts (this will refresh balances and wait 2 seconds per account)
print("Refreshing balances... (this takes a few seconds)\n")
accounts = await client.get_accounts(STRIPE_SESSION_ID)

if accounts:
    print("✅ Success! Your accounts:\n")
    print("-" * 60)
    
    total = 0
    for acc in accounts:
        print(f"\n💰 {acc.name}")
        print(f"   Balance: ${acc.balance:,.2f}")
        print(f"   Type: {acc.type.title()}")
        print(f"   Bank: {acc.institution_name}")
        total += acc.balance
    
    print("\n" + "=" * 60)
    print(f"Total: ${total:,.2f}")
    print("=" * 60)
    
    print("\n🎉 Perfect! Now you can use money_manager_demo.ipynb!")
else:
    print("❌ No accounts found. Check your session ID.")

## All Done!

**Next steps:**
1. Make sure `STRIPE_SESSION_ID` is saved in `secrets.env`
2. Open `money_manager_demo.ipynb` to start using the Money Manager!

**Your test accounts:**
- Connected via Stripe Financial Connections
- Real Stripe test data (balances, transactions)
- Ready for the Money Manager agent!